# Architectures

> Scripts to get architectures for the models

In [ ]:
#| default_exp architectures

In [ ]:
#| export
#| hide

import torch
import torch.nn as nn
from torch import Tensor
from pytorch_lightning.callbacks import Callback
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

## Sampling

In [ ]:
#| export
class Sampling(nn.Module):
    def forward(self, z_mean: Tensor, z_log_var: Tensor) -> Tensor:
        std = torch.exp(0.5 * z_log_var)
        eps = torch.randn_like(std)
        return z_mean + eps * std

## Callback

In [ ]:
#| export
class VAELossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.train_total_losses = []
        self.train_recon_losses = []
        self.train_kl_losses = []
        self.val_total_losses = []
        self.val_recon_losses = []
        self.val_kl_losses = []

    def on_train_epoch_end(self, trainer, pl_module):
        train_total_loss = trainer.callback_metrics.get('train_total_loss')
        train_recon_loss = trainer.callback_metrics.get('train_reconstruction_loss')
        train_kl_loss = trainer.callback_metrics.get('train_kl_loss')

        if train_total_loss is not None:
            self.train_total_losses.append(train_total_loss.item())
        if train_recon_loss is not None:
            self.train_recon_losses.append(train_recon_loss.item())
        if train_kl_loss is not None:
            self.train_kl_losses.append(train_kl_loss.item())

    def on_validation_epoch_end(self, trainer, pl_module):
        val_total_loss = trainer.callback_metrics.get('val_total_loss')
        val_recon_loss = trainer.callback_metrics.get('val_reconstruction_loss')
        val_kl_loss = trainer.callback_metrics.get('val_kl_loss')

        if val_total_loss is not None:
            self.val_total_losses.append(val_total_loss.item())
        if val_recon_loss is not None:
            self.val_recon_losses.append(val_recon_loss.item())
        if val_kl_loss is not None:
            self.val_kl_losses.append(val_kl_loss.item())

    def plot_total_losses(self, save_path=None):
        plt.figure(figsize=(10, 5))
        plt.plot(self.train_total_losses, label='Training Total Loss')
        plt.plot(self.val_total_losses, label='Validation Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Total Training and Validation Losses')
        plt.legend()
        plt.grid(True)
        if save_path:
            plt.savefig(save_path)
        plt.show()

    def plot_all_losses(self, save_path=None):
        plt.figure(figsize=(15, 10))
        
        plt.subplot(3, 1, 1)
        plt.plot(self.train_total_losses, label='Training Total Loss')
        plt.plot(self.val_total_losses, label='Validation Total Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Total Loss')
        plt.title('Total Training and Validation Losses')
        
        plt.subplot(3, 1, 2)
        plt.plot(self.train_recon_losses, label='Training Reconstruction Loss')
        plt.plot(self.val_recon_losses, label='Validation Reconstruction Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Reconstruction Loss')
        
        plt.subplot(3, 1, 3)
        plt.plot(self.train_kl_losses, label='Training KL Divergence Loss')
        plt.plot(self.val_kl_losses, label='Validation KL Divergence Loss')
        plt.xlabel('Epoch')
        plt.ylabel('KL Divergence Loss')
        
        for i in range(1, 4):
            plt.subplot(3, 1, i)
            plt.legend()
            plt.grid(True)

        plt.tight_layout()
        if save_path:
            plt.savefig(save_path)
        plt.show()

## VAEs Encoders and Decoders

In [ ]:
#| export
class VAEEncoder(ABC, nn.Module):
    def __init__(self, latent_dim: int):
        super(VAEEncoder, self).__init__()
        self.latent_dim = latent_dim

    @abstractmethod
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Encodes the input tensor into mean and log variance tensors.

        :param x: Input tensor of shape (batch_size, feat_dim, seq_len)
        :return: Tuple containing mean and log variance tensors.
        """
        pass

    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        return self.encode(x)

In [ ]:
#| export
class VAEDecoder(ABC, nn.Module):
    def __init__(self, latent_dim: int):
        super(VAEDecoder, self).__init__()
        self.latent_dim = latent_dim

    @abstractmethod
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """
        Decodes the latent tensor back to the original data space.

        :param z: Latent tensor of shape (batch_size, latent_dim)
        :return: Reconstructed tensor of shape (batch_size, feat_dim, seq_len)
        """
        pass

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        return self.decode(z)

### Simple Convolutional Architecture

#### Kernel Sizes: 5, 7, 9, 13

##### Encoder

In [ ]:
#| export
class Conv5Encoder(VAEEncoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)
        )

    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

##### Decoder

In [ ]:
#| export
class Conv5Decoder(VAEDecoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, self.seq_len))
        )

        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=13, stride=1, padding=6),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=9, stride=1, padding=4),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(p=self.dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

##### Getter

In [ ]:
#| export
def get_conv5_vae_components(seq_len: int,  # Length of input sequence
                           feat_dim: int,  # Dimensionality of input features 
                           latent_dim: int,  # Dimensionality of the latent space
                           dropout_rate: float = 0.2,  # Dropout rate for regularization
                           ) -> tuple[Conv5Encoder, Conv5Decoder]:
    """
    Creates and returns encoder and decoder components for a convolutional VAE architecture.
    """
    encoder = Conv5Encoder(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5Decoder(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

#### Legit Tsgm

##### Encoder

In [ ]:
#| export
class Conv5EncoderLegitTsgm(VAEEncoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        
        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Flatten()
        )
        
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )

    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

##### Decoder

In [ ]:
#| export
class Conv5DecoderLegitTsgm(VAEDecoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        # Dense layers to upscale the latent dimensions
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * seq_len),  # Adjust output size based on sequence length
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, seq_len))
        )

        # Transpose Convolutional layers with calculated padding for "same" effect
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=11, stride=1, padding=5),
            nn.Sigmoid()
        )

    def decode(self, z: Tensor) -> Tensor:
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

##### Getter

In [ ]:
#| export
def get_conv5_legit_tsgm_vae_components(seq_len: int,  # Length of input sequence
                                       feat_dim: int,  # Dimensionality of input features
                                       latent_dim: int,  # Dimensionality of the latent space
                                       dropout_rate: float = 0.2,  # Dropout rate for regularization
                                       ) -> tuple[Conv5EncoderLegitTsgm, Conv5DecoderLegitTsgm]:
    """
    Creates and returns encoder and decoder components for a Conv5 VAE model.
    """
    encoder = Conv5EncoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    decoder = Conv5DecoderLegitTsgm(seq_len, feat_dim, latent_dim, dropout_rate)
    
    return encoder, decoder

### Inception Time

The implemetation in the following cell is taken from
https://github.com/TheMrGhostman/InceptionTime-Pytorch/blob/master/inception.py , the next cell is an adjustment for our problem

In [ ]:
#| export
def correct_sizes(sizes):
	corrected_sizes = [s if s % 2 != 0 else s - 1 for s in sizes]
	return corrected_sizes


def pass_through(X):
	return X


class Inception(nn.Module):
	def __init__(self, in_channels, n_filters, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU(), return_indices=False):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		: param return_indices			Indices are needed only if we want to create decoder with InceptionTranspose with MaxUnpool1d. 
		"""
		super(Inception, self).__init__()
		self.return_indices=return_indices
		if in_channels > 1:
			self.bottleneck = nn.Conv1d(
								in_channels=in_channels, 
								out_channels=bottleneck_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		else:
			self.bottleneck = pass_through
			bottleneck_channels = 1

		self.conv_from_bottleneck_1 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_from_bottleneck_2 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_from_bottleneck_3 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.max_pool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1, return_indices=return_indices)
		self.conv_from_maxpool = nn.Conv1d(
									in_channels=in_channels, 
									out_channels=n_filters, 
									kernel_size=1, 
									stride=1,
									padding=0, 
									bias=False
									)
		self.batch_norm = nn.BatchNorm1d(num_features=4*n_filters)
		self.activation = activation

	def forward(self, X):
		# step 1
		Z_bottleneck = self.bottleneck(X)
		if self.return_indices:
			Z_maxpool, indices = self.max_pool(X)
		else:
			Z_maxpool = self.max_pool(X)
		# step 2
		Z1 = self.conv_from_bottleneck_1(Z_bottleneck)
		Z2 = self.conv_from_bottleneck_2(Z_bottleneck)
		Z3 = self.conv_from_bottleneck_3(Z_bottleneck)
		Z4 = self.conv_from_maxpool(Z_maxpool)
		# step 3 
		Z = torch.cat([Z1, Z2, Z3, Z4], axis=1)
		Z = self.activation(self.batch_norm(Z))
		if self.return_indices:
			return Z, indices
		else:
			return Z


class InceptionBlock(nn.Module):
	def __init__(self, in_channels, n_filters=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU(), return_indices=False):
		super(InceptionBlock, self).__init__()
		self.use_residual = use_residual
		self.return_indices = return_indices
		self.activation = activation
		self.inception_1 = Inception(
							in_channels=in_channels,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_2 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_3 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)	
		if self.use_residual:
			self.residual = nn.Sequential(
								nn.Conv1d(
									in_channels=in_channels, 
									out_channels=4*n_filters, 
									kernel_size=1,
									stride=1,
									padding=0
									),
								nn.BatchNorm1d(
									num_features=4*n_filters
									)
								)

	def forward(self, X):
		if self.return_indices:
			Z, i1 = self.inception_1(X)
			Z, i2 = self.inception_2(Z)
			Z, i3 = self.inception_3(Z)
		else:
			Z = self.inception_1(X)
			Z = self.inception_2(Z)
			Z = self.inception_3(Z)
		if self.use_residual:
			Z = Z + self.residual(X)
			Z = self.activation(Z)
		if self.return_indices:
			return Z,[i1, i2, i3]
		else:
			return Z

In [ ]:
#| export
class InceptionWithoutPool(nn.Module):
	def __init__(self, in_channels, n_filters, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU()):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		"""
		super(InceptionWithoutPool, self).__init__()
		if in_channels > 1:
			self.bottleneck = nn.Conv1d(
								in_channels=in_channels, 
								out_channels=bottleneck_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		else:
			self.bottleneck = pass_through
			bottleneck_channels = 1

		self.conv_from_bottleneck_1 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_from_bottleneck_2 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_from_bottleneck_3 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.batch_norm = nn.BatchNorm1d(num_features=3*n_filters)
		self.activation = activation

	def forward(self, X):
		# step 1
		Z_bottleneck = self.bottleneck(X)
		# step 2
		Z1 = self.conv_from_bottleneck_1(Z_bottleneck)
		Z2 = self.conv_from_bottleneck_2(Z_bottleneck)
		Z3 = self.conv_from_bottleneck_3(Z_bottleneck)
		# step 3 
		Z = torch.cat([Z1, Z2, Z3], axis=1)
		Z = self.activation(self.batch_norm(Z))
		return Z


class InceptionBlockWithoutPool(nn.Module):
    def __init__(self, in_channels, n_filters=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU(), return_indices=False):
        super(InceptionBlockWithoutPool, self).__init__()
        self.use_residual = use_residual
        self.activation = activation
        self.inception_1 = InceptionWithoutPool(
            in_channels=in_channels,
            n_filters=n_filters,
            kernel_sizes=kernel_sizes,
            bottleneck_channels=bottleneck_channels,
            activation=activation
        )
        self.inception_2 = InceptionWithoutPool(
            in_channels=3 * n_filters,
            n_filters=n_filters,
            kernel_sizes=kernel_sizes,
            bottleneck_channels=bottleneck_channels,
            activation=activation
        )
        self.inception_3 = InceptionWithoutPool(
            in_channels=3 * n_filters,
            n_filters=n_filters,
            kernel_sizes=kernel_sizes,
            bottleneck_channels=bottleneck_channels,
            activation=activation
        )   
        if self.use_residual:
            self.residual = nn.Sequential(
                nn.Conv1d(
                    in_channels=in_channels, 
                    out_channels=3 * n_filters, 
                    kernel_size=1,
                    stride=1,
                    padding=0
                ),
                nn.BatchNorm1d(
                    num_features=3 * n_filters
                )
            )

    def forward(self, X):
        Z = self.inception_1(X)
        Z = self.inception_2(Z)
        Z = self.inception_3(Z)

        if self.use_residual:
            Z = Z + self.residual(X)
            Z = self.activation(Z)
        return Z


class InceptionTransposeWithoutPool(nn.Module):
	def __init__(self, in_channels, out_channels, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU()):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		"""
		super(InceptionTransposeWithoutPool, self).__init__()
		self.activation = activation
		self.conv_to_bottleneck_1 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_to_bottleneck_2 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_to_bottleneck_3 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.bottleneck = nn.Conv1d(
								in_channels=3*bottleneck_channels, 
								out_channels=out_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		self.batch_norm = nn.BatchNorm1d(num_features=out_channels)

	def forward(self, X):
		Z1 = self.conv_to_bottleneck_1(X)
		Z2 = self.conv_to_bottleneck_2(X)
		Z3 = self.conv_to_bottleneck_3(X)

		Z = torch.cat([Z1, Z2, Z3], axis=1)
		BN = self.bottleneck(Z)
		
		return self.activation(self.batch_norm(BN))


class InceptionTransposeBlockWithoutPool(nn.Module):
	def __init__(self, in_channels, out_channels=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU()):
		super(InceptionTransposeBlockWithoutPool, self).__init__()
		self.use_residual = use_residual
		self.activation = activation
		self.inception_1 = InceptionTransposeWithoutPool(
							in_channels=in_channels,
							out_channels=in_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)
		self.inception_2 = InceptionTransposeWithoutPool(
							in_channels=in_channels,
							out_channels=in_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)
		self.inception_3 = InceptionTransposeWithoutPool(
							in_channels=in_channels,
							out_channels=out_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)	
		if self.use_residual:
			self.residual = nn.Sequential(
								nn.ConvTranspose1d(
									in_channels=in_channels, 
									out_channels=out_channels, 
									kernel_size=1,
									stride=1,
									padding=0
									),
								nn.BatchNorm1d(
									num_features=out_channels
									)
								)

	def forward(self, X):
		Z = self.inception_1(X)
		Z = self.inception_2(Z)
		Z = self.inception_3(Z)
		if self.use_residual:
			Z = Z + self.residual(X)
			Z = self.activation(Z)
		return Z

#### Encoder

In [ ]:
#| export
class InceptionTimeVAEEncoder(VAEEncoder):
    def __init__(self, feat_dim=7, seq_len=100, n_filters=32, kernel_sizes=[5, 11, 23],
                 bottleneck_channels=32, latent_dim=2):
        super(InceptionTimeVAEEncoder, self).__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        
        self.inception_blocks = nn.ModuleList([
            InceptionBlock(
                in_channels=feat_dim,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            ),
            InceptionBlock(
                in_channels=4 * n_filters,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            ),
            InceptionBlock(
                in_channels=4 * n_filters,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            )
        ])

        self.flatten = nn.Flatten()

        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=4 * n_filters * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)
        )
        
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, list]:
        indices_list = []
        for block in self.inception_blocks:
            x, indices = block(x)
            indices_list.append(indices)
        
        x = self.flatten(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        
        return z_mean, z_log_var, indices_list

In [ ]:
#| export
class WPInceptionTimeVAEEncoder(VAEEncoder):
    def __init__(self, feat_dim=7, seq_len=100, n_filters=32, kernel_sizes=[5, 11, 23],
                 bottleneck_channels=32, latent_dim=2):
        super(WPInceptionTimeVAEEncoder, self).__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim
        
        self.inception_blocks = nn.ModuleList([
            InceptionBlockWithoutPool(
                in_channels=feat_dim,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            ),
            InceptionBlockWithoutPool(
                in_channels=3 * n_filters,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            ),
            InceptionBlockWithoutPool(
                in_channels=3 * n_filters,
                n_filters=n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                use_residual=True,
                activation=nn.ReLU(),
                return_indices=True
            )
        ])

        self.flatten = nn.Flatten()

        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=3 * n_filters * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)
        )
        
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, list]:
        for block in self.inception_blocks:
            x = block(x)
        x = self.flatten(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        
        return z_mean, z_log_var

#### Decoder

In [ ]:
#| export
class WPInceptionTimeVAEDecoder(VAEDecoder):
    def __init__(self, feat_dim=7, seq_len=100, n_filters=32, kernel_sizes=[5, 11, 23],
                 bottleneck_channels=32, latent_dim=2):
        super(WPInceptionTimeVAEDecoder, self).__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.latent_dim = latent_dim

        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=3 * n_filters * self.seq_len),
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(3 * n_filters, self.seq_len))
        )

        self.inception_transpose_blocks = nn.ModuleList([
            InceptionTransposeBlockWithoutPool(
                in_channels=3 * n_filters,
                out_channels=3 * n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                activation=nn.ReLU()
            ),
            InceptionTransposeBlockWithoutPool(
                in_channels=3 * n_filters,
                out_channels=3 * n_filters,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                activation=nn.ReLU()
            ),
            InceptionTransposeBlockWithoutPool(
                in_channels=3 * n_filters,
                out_channels=feat_dim,
                kernel_sizes=kernel_sizes,
                bottleneck_channels=bottleneck_channels,
                activation=nn.ReLU()
            )
        ])
        
    def decode(self, z: torch.Tensor) -> torch.Tensor:
        z = self.dense_layers(z)
        for block in self.inception_transpose_blocks:
            z = block(z)
        
        return z

#### Getter

In [ ]:
#| export
def get_inception_time_vae_components(seq_len: int,  # Length of input sequence
                                    feat_dim: int,  # Dimensionality of input features 
                                    latent_dim: int,  # Dimensionality of the latent space
                                    without_pooling: bool = True,  # If True, returns WPInceptionTimeVAEEncoder instead of InceptionTimeVAEEncoder
                                    **model_kwargs: dict  # Dictionary containing model-specific keyword arguments
                                    ) -> tuple[Union[InceptionTimeVAEEncoder, WPInceptionTimeVAEEncoder], WPInceptionTimeVAEDecoder]:
    """
    Returns encoder and decoder components for an InceptionTime-based VAE architecture.
    """
    if without_pooling:
        encoder = WPInceptionTimeVAEEncoder(
            feat_dim=feat_dim,
            seq_len=seq_len,
            n_filters=model_kwargs.get('n_filters', 32),
            kernel_sizes=model_kwargs.get('kernel_sizes', [5, 11, 23]),
            bottleneck_channels=model_kwargs.get('bottleneck_channels', 32),
            latent_dim=latent_dim
        )
    else:
        encoder = InceptionTimeVAEEncoder(
            feat_dim=feat_dim,
            seq_len=seq_len,
            n_filters=model_kwargs.get('n_filters', 32),
            kernel_sizes=model_kwargs.get('kernel_sizes', [5, 11, 23]),
            bottleneck_channels=model_kwargs.get('bottleneck_channels', 32),
            latent_dim=latent_dim
        )
    
    decoder = WPInceptionTimeVAEDecoder(
        feat_dim=feat_dim,
        seq_len=seq_len,
        n_filters=model_kwargs.get('n_filters', 32),
        kernel_sizes=model_kwargs.get('kernel_sizes', [5, 11, 23]),
        bottleneck_channels=model_kwargs.get('bottleneck_channels', 32),
        latent_dim=latent_dim
    )
    
    return encoder, decoder

## cVAEs Encoders and Decoders

In [ ]:
#| export
class cVAEEncoder(ABC, nn.Module):
    """
    Abstract base for a conditional VAE encoder:
    Encodes data + condition into z_mean, z_log_var
    """
    def __init__(self, latent_dim: int):
        super().__init__()
        self.latent_dim = latent_dim

    @abstractmethod
    def encode(self, x: Tensor, cond: Tensor) -> tuple[Tensor, Tensor]:
        """
        Encodes the input tensor + condition into mean and log variance.
        """
        pass

    def forward(self, x: Tensor, cond: Tensor) -> tuple[Tensor, Tensor]:
        return self.encode(x, cond)

In [ ]:
#| export
class cVAEDecoder(ABC, nn.Module):
    """
    Abstract base for a conditional VAE decoder:
    Decodes z + condition into reconstructed data
    """
    def __init__(self, latent_dim: int):
        super().__init__()
        self.latent_dim = latent_dim

    @abstractmethod
    def decode(self, z: Tensor, cond: Tensor) -> Tensor:
        """
        Decodes the latent tensor + condition back to the original data space.
        """
        pass

    def forward(self, z: Tensor, cond: Tensor) -> Tensor:
        return self.decode(z, cond)

### Simple Convolutions

In [ ]:
#| export
class cConv5EncoderLegitTsgm(cVAEEncoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, cond_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        self.cond_dim = cond_dim

        self.convo_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.feat_dim + self.cond_dim, out_channels=64, kernel_size=10, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=4, stride=1, padding='same'),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.Flatten()
        )

        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=64 * self.seq_len, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.latent_dim * 2)  # Output z_mean and z_log_var
        )

    def encode(self, x: Tensor, cond: Tensor) -> tuple[Tensor, Tensor]:
        # Concatenate condition along feature dimension
        # cond shape: (batch_size, cond_dim). Expand to match (batch_size, cond_dim, seq_len)
        # If cond is shape (N,), do this first:
        if cond.ndim == 1:
            cond = cond.unsqueeze(1)  # now shape is [N, 1]

        cond = cond.unsqueeze(2).repeat(1, 1, x.shape[2])
        x = torch.cat([x, cond], dim=1)  # (batch_size, feat_dim + cond_dim, seq_len)
        
        x = self.convo_layers(x)
        x = self.dense_layers(x)
        z_mean, z_log_var = torch.split(x, self.latent_dim, dim=1)
        return z_mean, z_log_var

In [ ]:
#| export
class cConv5DecoderLegitTsgm(cVAEDecoder):
    def __init__(self, seq_len: int, feat_dim: int, latent_dim: int, cond_dim: int, dropout_rate: float):
        super().__init__(latent_dim=latent_dim)
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.dropout_rate = dropout_rate
        self.cond_dim = cond_dim 
        
        # Dense layers to upscale the latent dimensions
        self.dense_layers = nn.Sequential(
            nn.Linear(in_features=self.latent_dim + self.cond_dim, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=64 * seq_len),  # Adjust output size based on sequence length
            nn.ReLU(),
            nn.Unflatten(dim=1, unflattened_size=(64, seq_len))
        )

        # Transpose Convolutional layers with "same" padding
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),
            nn.ConvTranspose1d(in_channels=64, out_channels=self.feat_dim, kernel_size=11, stride=1, padding=5),
            nn.Sigmoid()
        )
    def decode(self, z: Tensor, cond: Tensor) -> Tensor:
        # Concatenate condition with latent vector
        # cond shape: (batch_size, cond_dim)
        z = torch.cat([z, cond], dim=-1)  # (batch_size, latent_dim + cond_dim)
        
        z = self.dense_layers(z)
        z = self.conv_transpose_layers(z)
        return z

In [ ]:
#| export
def get_conditional_conv5_legit_tsgm_vae_components(
    seq_len: int,  # Length of input sequence
    feat_dim: int,  # Dimensionality of input features 
    latent_dim: int,  # Dimensionality of the latent space
    dropout_rate: float = 0.2,  # Dropout rate for regularization
    cond_dim: int = 1,  # Dimensionality of conditional input
) -> tuple[cConv5EncoderLegitTsgm, cConv5DecoderLegitTsgm]:
    """
    Creates encoder and decoder components for a conditional convolutional VAE.
    """
    encoder = cConv5EncoderLegitTsgm(seq_len, feat_dim, latent_dim, cond_dim, dropout_rate)
    decoder = cConv5DecoderLegitTsgm(seq_len, feat_dim, latent_dim, cond_dim, dropout_rate)
    
    return encoder, decoder

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()